In [94]:
import dgl.function as fn
import dgl 
import torch 
data_dict = {
    ('user', 'follows', 'user'): (torch.tensor([0, 1,2]), torch.tensor([1, 2,1])),
    ('user', 'like', 'topic'): (torch.tensor([1]), torch.tensor([1])),    
}
G = dgl.heterograph(data_dict)
print(G)

Graph(num_nodes={'topic': 2, 'user': 3},
      num_edges={('user', 'follows', 'user'): 3, ('user', 'like', 'topic'): 1},
      metagraph=[('user', 'user', 'follows'), ('user', 'topic', 'like')])


In [95]:
# 打印出每种类型的边，这会把每个类型的边都做一个总结输出，即不会有重复的（node_type,edge_type,node_type）
print(G.canonical_etypes) 

[('user', 'follows', 'user'), ('user', 'like', 'topic')]


In [97]:
'''给节点赋特征值
因为是异质图，所以这里的必须指定不同的节点赋特征值
'''
feat_user = torch.tensor([[ 2.2355, 1.0492],
         [ 0.5203,  -1.0647],
         [ -0.6342, -0.5373]])
feat_topic = torch.tensor([[2.3238],
         [0.5637]])
G.ndata['x'] = {'user': feat_user, 'topic': feat_topic}
print(G.ndata['x'])

# 针对不同的边类型进行赋值data
follows_weight = torch.tensor([0.3,0.5,0.1]) # 个数要和边数一致
like_weight = torch.tensor([0.2])
G.edges['follows'].data['w'] = follows_weight  
G.edges['like'].data['w'] = like_weight  # 存储边权
print(G.edata)

{'topic': tensor([[2.3238],
        [0.5637]]), 'user': tensor([[ 2.2355,  1.0492],
        [ 0.5203, -1.0647],
        [-0.6342, -0.5373]])}
defaultdict(<class 'dict'>, {'w': {('user', 'follows', 'user'): tensor([0.3000, 0.5000, 0.1000]), ('user', 'like', 'topic'): tensor([0.2000])}})


In [98]:
import dgl.function as fn

# 每种关系的表示是一个矩阵，也就是w_r
# 关系矩阵的size是(2,3)，所以经过RGCN 后，每个节点的特征就从2维变成了3维
follow = torch.tensor([[ 1.2812,  0.8862,  0.2033],
        [ 0.4951,  0.8404, -0.2751]]) 
like = torch.tensor([[-0.1666, -0.7685,  2.7836],
        [ 1.0682, -0.4773,  0.7841]])
weight = {'follows':follow,'like':like} # 
funcs = {}
for c_etype in G.canonical_etypes:
    srctype, etype, dsttype = c_etype
    print(srctype,etype,dsttype)
    print(G.ndata['x'][srctype]) # 获取到节点对应的特征 (3,2)
    print(weight[etype]) # (2,3)
    # 该边的特征 * 原节点的特征， 得到每个待传递的消息
    # G.ndata['x'][srctype] 就相当于将所有srctype 都乘了这个关系矩阵，尽管这个节点可能没有这条边
    Wh = (G.ndata['x'][srctype]) @ weight[etype]  
    print("Wh:",Wh) 
    
    # 把它存在图中用来做消息传递
    # 指定类型 srctype, 再指定特征类型 'Wh_etype'
    G.nodes[srctype].data['Wh_%s' % etype] = Wh    
    
    # 使用一个字典指定每个关系的消息传递函数：(message_func, reduce_func). 
    # 注意结果保存在同一个目标特征“h”，说明聚合是逐类进行的。
#     funcs[etype] = (fn.copy_u('Wh_%s' % etype, 'm'), fn.mean('m', 'h'))
    # 下面的这个 fn.mean('m','h') 是对所有边类型的数据做一个mean操作，针对的是同一个类型
    funcs[etype] = (fn.u_mul_e('Wh_%s' % etype,'w', 'm'), fn.mean('m', 'h'))

# 将每个类型消息聚合的结果相加，针对的是不同类型
G.multi_update_all(funcs, 'sum')

# 返回更新过的节点特征字典
out= {ntype : G.nodes[ntype].data['h'] for ntype in G.ntypes}
print(out)

user follows user
tensor([[ 2.2355,  1.0492],
        [ 0.5203, -1.0647],
        [-0.6342, -0.5373]])
tensor([[ 1.2812,  0.8862,  0.2033],
        [ 0.4951,  0.8404, -0.2751]])
Wh: tensor([[ 3.3836,  2.8628,  0.1658],
        [ 0.1395, -0.4337,  0.3987],
        [-1.0786, -1.0136,  0.0189]])
user like topic
tensor([[ 2.2355,  1.0492],
        [ 0.5203, -1.0647],
        [-0.6342, -0.5373]])
tensor([[-0.1666, -0.7685,  2.7836],
        [ 1.0682, -0.4773,  0.7841]])
Wh: tensor([[ 0.7483, -2.2188,  7.0454],
        [-1.2240,  0.1083,  0.6135],
        [-0.4683,  0.7438, -2.1867]])
{'topic': tensor([[ 0.0000,  0.0000,  0.0000],
        [-0.2448,  0.0217,  0.1227]]), 'user': tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.4536,  0.3787,  0.0258],
        [ 0.0697, -0.2168,  0.1993]])}


In [104]:
a = torch.tensor([3.3835,2.8628,0.1658])
b =torch.tensor([-1.0786,-1.0136,0.0189])
c = a * 0.3 + b * 0.1
print(c/2)

tensor([0.4536, 0.3787, 0.0258])
